In [1]:
# ## 1. Introducción
#
# En el desarrollo de modelos de aprendizaje automático, especialmente con datos médicos, es fundamental asegurarse de que los conjuntos de datos utilizados para entrenamiento, validación y prueba sean independientes. Una forma común de **fuga de datos** (data leakage) ocurre cuando información del conjunto de prueba o validación "se filtra" al conjunto de entrenamiento.
#
# Una fuente específica de fuga en datos médicos es la **superposición de pacientes**. Si un mismo paciente tiene imágenes tanto en el conjunto de entrenamiento como en el de validación o prueba, el modelo podría aprender características específicas de ese paciente durante el entrenamiento y luego obtener una ventaja injusta al evaluarlo en las imágenes del mismo paciente en los otros conjuntos. Esto lleva a una sobreestimación del rendimiento real del modelo en datos completamente nuevos.
#
# **Objetivos:**
# * Identificar si existe superposición de pacientes entre los conjuntos de entrenamiento, validación y prueba.
# * Cuantificar la superposición.
# * Demostrar cómo eliminar la superposición para prevenir la fuga de datos.

# ## 2. Importar Librerías y Cargar Datos

# In[ ]:
import pandas as pd
import numpy as np
import os


In [2]:
# In[ ]:
# Define paths (ajusta si es necesario basado en la ubicación del notebook)
BASE_DATA_PATH = "../data/"
train_csv_path = os.path.join(BASE_DATA_PATH, "train-small.csv")
valid_csv_path = os.path.join(BASE_DATA_PATH, "valid-small.csv")
test_csv_path = os.path.join(BASE_DATA_PATH, "test.csv")

# Columna que contiene el ID del paciente
PATIENT_ID_COL = 'PatientId'

# Load the dataframes
try:
    train_df = pd.read_csv(train_csv_path)
    valid_df = pd.read_csv(valid_csv_path)
    test_df = pd.read_csv(test_csv_path)
    print("Archivos CSV cargados exitosamente.")
    print(f"Tamaño Entrenamiento: {train_df.shape}")
    print(f"Tamaño Validación: {valid_df.shape}")
    print(f"Tamaño Prueba: {test_df.shape}")
except FileNotFoundError as e:
    print(f"Error: No se encontró el archivo CSV: {e}")
    print(f"Asegúrate de que los archivos estén en la carpeta '{BASE_DATA_PATH}'")
    raise e
except Exception as e:
    print(f"Ocurrió un error al cargar los datos: {e}")
    raise e

Archivos CSV cargados exitosamente.
Tamaño Entrenamiento: (1000, 16)
Tamaño Validación: (200, 16)
Tamaño Prueba: (420, 16)


In [7]:
## 3. Extraer y Comparar IDs de Pacientes

# Extraeremos los identificadores únicos de pacientes de cada conjunto y usaremos conjuntos (`set`) para encontrar eficientemente las intersecciones (pacientes presentes en más de un conjunto).

# In[ ]:
# Verificar que la columna de ID de paciente exista
if PATIENT_ID_COL not in train_df.columns or \
   PATIENT_ID_COL not in valid_df.columns or \
   PATIENT_ID_COL not in test_df.columns:
    print(f"Error: La columna '{PATIENT_ID_COL}' no se encuentra en todos los DataFrames.")
    # Detener ejecución o manejar el error
    raise KeyError(f"Columna '{PATIENT_ID_COL}' faltante.")

# Extraer IDs únicos de pacientes de cada conjunto
# Usar .astype(str) sigue siendo una buena práctica por robustez
train_patient_ids = set(train_df[PATIENT_ID_COL].astype(str).unique())
valid_patient_ids = set(valid_df[PATIENT_ID_COL].astype(str).unique())
test_patient_ids = set(test_df[PATIENT_ID_COL].astype(str).unique())

print(f"Número de pacientes únicos en Entrenamiento: {len(train_patient_ids)}")
print(f"Número de pacientes únicos en Validación: {len(valid_patient_ids)}")
print(f"Número de pacientes únicos en Prueba: {len(test_patient_ids)}")


Número de pacientes únicos en Entrenamiento: 928
Número de pacientes únicos en Validación: 199
Número de pacientes únicos en Prueba: 389


In [8]:
# Calcular intersecciones para identificar superposición
overlap_train_valid = train_patient_ids.intersection(valid_patient_ids)
overlap_train_test = train_patient_ids.intersection(test_patient_ids)
overlap_valid_test = valid_patient_ids.intersection(test_patient_ids)

print(f"\n--- Análisis de Superposición de Pacientes ---")
print(f"Pacientes en Entrenamiento Y Validación: {len(overlap_train_valid)}") # Ahora debería dar 11
if len(overlap_train_valid) > 0:
    print(f"   IDs de ejemplo: {list(overlap_train_valid)[:15]}...") # Mostrar algunos IDs (hasta 15)

print(f"\nPacientes en Entrenamiento Y Prueba: {len(overlap_train_test)}")
if len(overlap_train_test) > 0:
    print(f"   IDs de ejemplo: {list(overlap_train_test)[:5]}...")

print(f"\nPacientes en Validación Y Prueba: {len(overlap_valid_test)}")
if len(overlap_valid_test) > 0:
    print(f"   IDs de ejemplo: {list(overlap_valid_test)[:5]}...")

# Comprobar si hay algún paciente en los tres conjuntos
overlap_all_three = train_patient_ids.intersection(valid_patient_ids).intersection(test_patient_ids)
print(f"\nPacientes en Entrenamiento Y Validación Y Prueba: {len(overlap_all_three)}")



--- Análisis de Superposición de Pacientes ---
Pacientes en Entrenamiento Y Validación: 197
   IDs de ejemplo: ['14125', '8130', '12615', '16030', '8348', '6426', '13904', '28550', '10651', '7523', '22135', '29454', '14107', '15078', '12640']...

Pacientes en Entrenamiento Y Prueba: 0

Pacientes en Validación Y Prueba: 0

Pacientes en Entrenamiento Y Validación Y Prueba: 0


In [9]:
 ## 4. Eliminar Superposición de Pacientes

# Si se detecta superposición, es crucial eliminarla para evitar la fuga de datos. Una estrategia común es **eliminar los pacientes superpuestos de los conjuntos de validación y prueba**, manteniendo intacto el conjunto de entrenamiento. Esto asegura que la evaluación se realice sobre pacientes completamente "no vistos" durante el entrenamiento.

# In[ ]:
print("\n--- Eliminando Superposición (de Validación y Prueba) ---")

# Convertir los conjuntos de IDs superpuestos a listas para usarlos con .isin()
# Usamos los conjuntos recalculados con el nombre de columna correcto
patients_to_remove_from_valid = list(overlap_train_valid.union(overlap_valid_test))
patients_to_remove_from_test = list(overlap_train_test.union(overlap_valid_test))
# Nota: Incluimos overlap_valid_test en ambos para asegurarnos de que se eliminen de ambos si existe.

print(f"Pacientes a eliminar de Validación (presentes en Entrenamiento o Prueba): {len(patients_to_remove_from_valid)}")
print(f"Pacientes a eliminar de Prueba (presentes en Entrenamiento o Validación): {len(patients_to_remove_from_test)}")

# Crear copias de los DataFrames originales antes de modificarlos (buena práctica)
valid_df_cleaned = valid_df.copy()
test_df_cleaned = test_df.copy()

# Filtrar: Mantener solo las filas donde el Patient ID NO está en la lista de eliminación
if patients_to_remove_from_valid:
    # Asegurarse de comparar como strings si los IDs en la lista son strings
    valid_df_cleaned = valid_df_cleaned[~valid_df_cleaned[PATIENT_ID_COL].astype(str).isin(patients_to_remove_from_valid)]
    print(f"\nTamaño del conjunto de Validación después de limpiar: {valid_df_cleaned.shape}")
else:
    print("\nNo se eliminaron pacientes del conjunto de Validación.")

if patients_to_remove_from_test:
    # Asegurarse de comparar como strings
    test_df_cleaned = test_df_cleaned[~test_df_cleaned[PATIENT_ID_COL].astype(str).isin(patients_to_remove_from_test)]
    print(f"Tamaño del conjunto de Prueba después de limpiar: {test_df_cleaned.shape}")
else:
    print("No se eliminaron pacientes del conjunto de Prueba.")


--- Eliminando Superposición (de Validación y Prueba) ---
Pacientes a eliminar de Validación (presentes en Entrenamiento o Prueba): 197
Pacientes a eliminar de Prueba (presentes en Entrenamiento o Validación): 0

Tamaño del conjunto de Validación después de limpiar: (2, 16)
No se eliminaron pacientes del conjunto de Prueba.


In [10]:
## 5. Comprobar la Eliminación de Superposición

# Verifiquemos que la superposición se haya eliminado correctamente entre los conjuntos limpios y el de entrenamiento.

# In[ ]:
print("\n--- Verificando Superposición Después de la Limpieza ---")

# Extraer IDs de los conjuntos limpios
valid_patient_ids_cleaned = set(valid_df_cleaned[PATIENT_ID_COL].astype(str).unique())
test_patient_ids_cleaned = set(test_df_cleaned[PATIENT_ID_COL].astype(str).unique())

# Recalcular intersecciones
overlap_train_valid_cleaned = train_patient_ids.intersection(valid_patient_ids_cleaned)
overlap_train_test_cleaned = train_patient_ids.intersection(test_patient_ids_cleaned)
overlap_valid_test_cleaned = valid_patient_ids_cleaned.intersection(test_patient_ids_cleaned)

print(f"Pacientes en Entrenamiento Y Validación (Limpiado): {len(overlap_train_valid_cleaned)}")
print(f"Pacientes en Entrenamiento Y Prueba (Limpiado): {len(overlap_train_test_cleaned)}")
print(f"Pacientes en Validación (Limpiado) Y Prueba (Limpiado): {len(overlap_valid_test_cleaned)}")

if len(overlap_train_valid_cleaned) == 0 and len(overlap_train_test_cleaned) == 0 and len(overlap_valid_test_cleaned) == 0:
    print("\n¡Éxito! Se ha eliminado la superposición de pacientes entre los conjuntos.")
else:
    print("\n¡Advertencia! Todavía existe superposición después de la limpieza. Revisa la lógica.")


# ## 6. Conclusión y Próximos Pasos

# Hemos identificado (correctamente ahora, esperamos) y eliminado con éxito la superposición de pacientes entre los conjuntos de entrenamiento, validación y prueba. Este es un paso esencial para garantizar que la evaluación del modelo sea representativa de su rendimiento en datos verdaderamente nuevos y no vistos.
#
# **Próximos Pasos:**
# * Utilizar los DataFrames **limpios** (`train_df`, `valid_df_cleaned`, `test_df_cleaned`) al crear los generadores de datos (`ImageDataGenerator` o `tf.data`) para el entrenamiento y la evaluación del modelo CNN.
# * Asegurarse de que cualquier división futura de datos (si se realiza) se haga a nivel de paciente para mantener la independencia de los conjuntos.



--- Verificando Superposición Después de la Limpieza ---
Pacientes en Entrenamiento Y Validación (Limpiado): 0
Pacientes en Entrenamiento Y Prueba (Limpiado): 0
Pacientes en Validación (Limpiado) Y Prueba (Limpiado): 0

¡Éxito! Se ha eliminado la superposición de pacientes entre los conjuntos.


In [11]:
print(valid_df_cleaned)

                Image  Atelectasis  Cardiomegaly  Consolidation  Edema  \
198  00003272_000.png            0             0              0      0   
199  00001108_004.png            1             0              0      0   

     Effusion  Emphysema  Fibrosis  Hernia  Infiltration  Mass  Nodule  \
198         0          0         0       1             0     0       0   
199         0          0         0       0             0     0       0   

     PatientId  Pleural_Thickening  Pneumonia  Pneumothorax  
198       3272                   0          0             0  
199       1108                   0          1             0  


In [12]:
print(valid_df_cleaned.shape)

(2, 16)
